<center><img src="./images/rtu_logo_lv.png" alt="RTU" title="RTU" width="150em"/></center>

<center><b>RĪGAS TEHNISKĀ UNIVERSITĀTE</b></center>
<center><b>DATORZINĀTNES UN INFORMĀCIJAS TEHNOLOĢIJAS FAKULTĀTE</b></center>
<center><b>Priekšmeta "Ievads mākslīgos neironu tīklos"</b></center><center>

<center><b>1. mājasdarbs</b></center>

<div style="text-align: right">
    LDsI 12300 Lietišķo datorsistēmu institūts<br>
    RDBD0 Datorsistēmas<br>
    <b>3. kursa 3. grupas</b> pilna laika klātienes <b>students</b><br>
    Oļegs Korsaks<br>
    <b>Studenta apliecības Nr.: </b>051RDB146<br>
</div>
<center><b>Rīga, 2021</b></center>
<div style="display:none">\pagebreak</div>

***

In [1]:
from math import exp
from operator import methodcaller

import numpy as np
import tensorflow as tf

In [2]:
letters = [
    ['A',[0,1,0,1,0,1,1,1,1,1,0,1,1,0,1],[1,0,0]],
    ['E',[1,1,1,1,0,0,1,1,0,1,0,0,1,1,1],[1,0,0]],
    ['D',[1,1,0,1,0,1,1,0,1,1,0,1,1,1,0],[1,0,0]],
    ['B',[1,1,0,1,0,1,1,1,0,1,0,1,1,1,0],[0,1,0]],
    ['C',[1,1,1,1,0,0,1,0,0,1,0,0,1,1,1],[0,1,0]],
    ['F',[1,1,1,1,0,0,1,1,0,1,0,0,1,0,0],[0,1,0]],
    ['G',[1,1,1,1,0,0,1,0,1,1,0,1,1,1,1],[0,0,1]],
    ['H',[1,0,1,1,0,1,1,1,1,1,0,1,1,0,1],[0,0,1]],
    ['I',[0,1,0,0,1,0,0,1,0,0,1,0,0,1,0],[0,0,1]],
]

In [3]:
def a_step(val):
    return int(val>0)


def a_lin(val):
    if val <= 0:
        return 0.0
    elif val > 5:
        return 1.0
    
    return val/5.0


def a_sigmoid(val):
    return 1.0 / (1.0+exp(-val))


def a_tanh(val):
    return (exp(val)-exp(-val)) / (exp(val)+exp(-val))

In [4]:
learning_rate = 0.1


class Neiron:
    def __init__(self, in_size):
        self.weights = np.zeros(in_size)
        self.w0 = 0

    def classify(self, incoming_signals):
        net = np.dot(incoming_signals, self.weights) + self.w0
        
        return a_step(net)
    
    def learn(self, incoming_signals, c):
        y = self.classify(incoming_signals)
        delta = c - y
        
        if delta:
            for i in range(len(self.weights)):
                self.weights[i] += delta * learning_rate * incoming_signals[i]

            self.w0 += delta * learning_rate

In [5]:
def run_neirons(data: list, neiron_class, in_size: int, class_count: int, epoch_count: int = 100):
    neirons = tuple(neiron_class(in_size) for _ in range(class_count))
    
    for epoch in range(epoch_count):
        for sample in data:
            for i, neiron in enumerate(neirons):
                neiron.learn(sample[1], sample[2][i])
    
    for sample in data:
        method = methodcaller('classify', sample[1])
        print(f'{sample[0]} {tuple(map(method, neirons))}')

In [6]:
run_neirons(letters, Neiron, 15, 3, 100)

A (1, 0, 0)
E (1, 0, 0)
D (1, 0, 0)
B (0, 1, 0)
C (0, 1, 0)
F (0, 1, 0)
G (0, 0, 1)
H (0, 0, 1)
I (0, 0, 1)


## Mājasdarbs 1

Pārveidosim neirona klasi, lai viņš varētu klasificēt 4 klašu simbolus. Pievienosim ! _ un . simbolus

In [7]:
letters = [
    ['A',[0,1,0,1,0,1,1,1,1,1,0,1,1,0,1],[1,0,0,0]],
    ['E',[1,1,1,1,0,0,1,1,0,1,0,0,1,1,1],[1,0,0,0]],
    ['D',[1,1,0,1,0,1,1,0,1,1,0,1,1,1,0],[1,0,0,0]],
    ['B',[1,1,0,1,0,1,1,1,0,1,0,1,1,1,0],[0,1,0,0]],
    ['C',[1,1,1,1,0,0,1,0,0,1,0,0,1,1,1],[0,1,0,0]],
    ['F',[1,1,1,1,0,0,1,1,0,1,0,0,1,0,0],[0,1,0,0]],
    ['G',[1,1,1,1,0,0,1,0,1,1,0,1,1,1,1],[0,0,1,0]],
    ['H',[1,0,1,1,0,1,1,1,1,1,0,1,1,0,1],[0,0,1,0]],
    ['I',[0,1,0,0,1,0,0,1,0,0,1,0,0,1,0],[0,0,1,0]],
    
    ['!',[0,1,0,0,1,0,0,1,0,0,0,0,0,1,0],[0,0,0,1]],
    ['_',[0,0,0,0,0,0,0,0,0,0,0,0,1,1,1],[0,0,0,1]],
    ['.',[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,1]],
]


class NeironHome:
    def __init__(self, in_size):
        self.weights = np.zeros(in_size)
        self.w0 = 0

    def classify(self, incoming_signals):
        net = np.dot(incoming_signals, self.weights) + self.w0
        
        return a_sigmoid(net)
    
    def learn(self, incoming_signals, c):
        y = self.classify(incoming_signals)
        delta = c - y
        
        if delta:
            for i in range(len(self.weights)):
                self.weights[i] += delta * learning_rate * incoming_signals[i]

            self.w0 += delta * learning_rate

In [8]:
run_neirons(letters, NeironHome, 15, 4, 1000)

A (0.9975535051097201, 6.660629571448497e-06, 0.008248493415553408, 0.00013232113048282796)
E (0.8293688905071558, 0.07107663617372244, 0.00345475745807826, 0.0008908342725369956)
D (0.8834289329308431, 0.040197189905192325, 0.005393751338105291, 0.000374360388021805)
B (0.06296538404255056, 0.9611077227614221, 2.5487150667717718e-05, 0.00044414160389372084)
C (0.08063124850588929, 0.941473518306242, 0.00588321544731504, 0.0016078235509806357)
F (0.02633476838322883, 0.9911138221082045, 0.0034084107149066696, 2.9723027415453927e-05)
G (0.04066958299013252, 0.00015968439826207983, 0.9849054420714136, 0.0002711874602893066)
H (0.02117992296299061, 1.3000067100018376e-05, 0.9923537292619337, 1.8435006117880236e-05)
I (0.0037253089873815924, 2.5566882336613036e-05, 0.9800763983367607, 0.02047026562186848)
! (0.018415486080804077, 4.3871158090628676e-05, 0.013755496597630701, 0.9781077632338467)
_ (0.00838609151522248, 0.00038552142987693667, 0.0020254630608827185, 0.9968259609187382)
. (0.

**Ir redzams, ka "!", "\_" un "." simboli tiek klasificēti kā 4. klase. Citu burtu klasifikācija nemainās.**

Pārtaisīsim TensorFlow kodu, lai viņš klasificētu burtus:

In [9]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(15, kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='sigmoid', input_shape=(15, )))
model.add(tf.keras.layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')

np_letters = np.array(tuple(np.array(letter[1]) for letter in letters))
targets = np.array(tuple(np.array(letter[2]) for letter in letters))
print(targets)

model.fit(np_letters, targets, epochs=1000, verbose=0)
predictions = model.predict(np_letters)

print(predictions)

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]
[[0.37047184 0.2304135  0.2934439  0.10567079]
 [0.2994195  0.36587617 0.20893149 0.12577288]
 [0.3621907  0.29258814 0.23302452 0.11219666]
 [0.35252976 0.37638938 0.16325071 0.10783017]
 [0.2886803  0.36144426 0.20185332 0.1480222 ]
 [0.29444072 0.41434592 0.17537645 0.1158369 ]
 [0.29668406 0.23614568 0.36185062 0.10531965]
 [0.2921281  0.20554838 0.41453084 0.08779269]
 [0.1139712  0.12026215 0.3248356  0.44093102]
 [0.11942177 0.13622987 0.23930153 0.50504684]
 [0.11129063 0.11937999 0.2033159  0.5660135 ]
 [0.08803236 0.10625695 0.1827992  0.62291145]]


**Interesanti**, bet TensorFlow versija apstiprināja manas šaubas par "I" un "!" lidzīguma problēmu, kāmēr bez TensorFlow klasifikācija nostrādāja pareizi. Ar parējiem burtiem arī ir redzamas problēmas - B tiek pieskaitīts pie 2. klases. Laikam vajag izmantot to pašu learning_rate:

In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(15, kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='sigmoid', input_shape=(15, )))
model.add(tf.keras.layers.Dense(4, activation='softmax'))
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

model.fit(np_letters, targets, epochs=1000, verbose=0)
predictions = model.predict(np_letters)

print(predictions)

[[9.7850138e-01 3.9678505e-03 1.7529514e-02 1.2821092e-06]
 [7.9518217e-01 1.7347169e-01 3.1345189e-02 9.0954171e-07]
 [7.7770329e-01 2.1985644e-01 2.4283789e-03 1.1892006e-05]
 [1.7629546e-01 8.2368726e-01 1.4823294e-05 2.4451660e-06]
 [3.0767524e-01 6.7438763e-01 1.7931392e-02 5.6735616e-06]
 [1.1542365e-01 8.7913316e-01 5.4004663e-03 4.2619431e-05]
 [7.2594583e-02 7.8612077e-04 9.2661911e-01 1.0930746e-07]
 [2.4757013e-02 1.3824074e-04 9.7510457e-01 1.7496119e-07]
 [3.3068235e-03 3.2525688e-05 9.9658835e-01 7.2217714e-05]
 [8.8739239e-02 8.4000407e-03 2.7733073e-01 6.2553006e-01]
 [3.0002363e-02 3.2191142e-02 3.0051593e-02 9.0775484e-01]
 [1.3530486e-04 8.8708272e-04 4.0348867e-04 9.9857414e-01]]


Tagad ir pariza atbilde. Bet darot 2. mājasdarbu, ieguvu nedaudz pieredzes un sapratu, ka pēc noklusējuma TensorFlow izmanto batch_size=32, bet oriģinālais modelis strādā batch_size=1 režīmā. Pārbaudīsim:

In [11]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(15, kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='sigmoid', input_shape=(15, )))
model.add(tf.keras.layers.Dense(4, activation='softmax'))
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

model.fit(np_letters, targets, batch_size=1, epochs=1000, verbose=0)
predictions = model.predict(np_letters)

print(predictions)

[[3.05631578e-01 1.51655555e-01 4.88562882e-01 5.41500151e-02]
 [3.86233449e-01 3.92386198e-01 2.17745125e-01 3.63524002e-03]
 [3.63157481e-01 2.43554294e-01 3.74557376e-01 1.87308919e-02]
 [3.85635704e-01 4.00453389e-01 2.10592687e-01 3.31821805e-03]
 [3.86552691e-01 3.86719853e-01 2.22852662e-01 3.87472543e-03]
 [3.79664510e-01 4.43765670e-01 1.74557656e-01 2.01221416e-03]
 [2.96097517e-01 1.41287014e-01 5.01036882e-01 6.15785606e-02]
 [1.05644405e-01 2.51650922e-02 4.91024464e-01 3.78166050e-01]
 [1.33664876e-01 3.56054790e-02 5.28038085e-01 3.02691609e-01]
 [2.43924633e-02 3.45864682e-03 2.41100103e-01 7.31048822e-01]
 [3.15344590e-03 2.56100378e-04 7.01030493e-02 9.26487446e-01]
 [1.52069435e-03 1.02617458e-04 4.42582332e-02 9.54118490e-01]]


Abos gadījumos atbilde ir pareiza, bet oriģinālā tomēr precīzāk sanāca.